In [19]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib notebook
import seaborn as sns

## Read data

In [20]:
df = pd.read_csv("./output/all-data.csv")

## Data Cleaning

#### Get concise summary of the dataframe 

In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 186850 entries, 0 to 186849
Data columns (total 6 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   Order ID          186305 non-null  object
 1   Product           186305 non-null  object
 2   Quantity Ordered  186305 non-null  object
 3   Price Each        186305 non-null  object
 4   Order Date        186305 non-null  object
 5   Purchase Address  186305 non-null  object
dtypes: object(6)
memory usage: 8.6+ MB


### First: Column labels

In [22]:
df.columns

Index(['Order ID', 'Product', 'Quantity Ordered', 'Price Each', 'Order Date',
       'Purchase Address'],
      dtype='object')

##### We notice many problems with column labels:
1. They are long.
2. There are spaces in each label.

#### Rename

In [23]:
df.rename(columns={"Quantity Ordered":"Quantity",
                   "Price Each":"unit price","Order Date":"Date",
                   "Purchase Address":"Address"},inplace=True)

#### Capitalize

In [24]:
df.rename(str.title, axis='columns',inplace=True)

#### Remove spaces 

In [25]:
df.columns=df.columns.str.replace(" ","")

##### Let's look at them now

In [26]:
df.columns

Index(['OrderId', 'Product', 'Quantity', 'UnitPrice', 'Date', 'Address'], dtype='object')